In [ ]:
from google import genai
from google.genai import types
import base64
from lib.story import system_prompt_v4

def generate():
  client = genai.Client(
      vertexai=True,
      project="thematic-land-451915-j3",
      location="us-central1",
  )

  si_text1 = system_prompt_v4

  model = "gemini-2.0-flash-001"
  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text="""mèo siêu anh hùng""")
      ]
    )
  ]
  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    system_instruction=[types.Part.from_text(text=si_text1)],
  )

  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    print(chunk.text, end="")

generate()

```json
{
  "summary": "Super Meo phải đối mặt với tiến sĩ Hamster độc ác, kẻ đang cố gắng biến tất cả mọi người thành chuột bằng một phát minh mới!",
  "title": "Super Meo và Tia Biến Hình",
  "characters": {
    "Super Meo": {
      "description": "Một chú mèo mướp dũng cảm với bộ lông màu cam và đôi mắt xanh lá cây sáng ngời, mặc một bộ đồ siêu anh hùng màu xanh dương.",
      "personality": "Dũng cảm, thông minh và luôn sẵn sàng giúp đỡ mọi người."
    },
    "Tiến sĩ Hamster": {
      "description": "Một con chuột hamster béo tròn với bộ lông xám và cặp kính lúp lớn, mặc một áo khoác phòng thí nghiệm trắng.",
      "personality": "Xảo quyệt, ích kỷ và luôn tìm cách để thống trị thế giới."
    },
    "Lily": {
      "description": "Một cô bé có mái tóc vàng và đôi mắt to tròn, luôn mặc một chiếc váy hoa.",
      "personality": "Tò mò, tốt bụng và là bạn thân của Super Meo."
    }
  },
  "pages": [
    {
      "page": 1,
      "scene": "Super Meo đang tuần tra thành phố trên nóc nhà

In [ ]:
client = genai.Client(
      vertexai=True,
      project="thematic-land-451915-j3",
      location="us-central1",
  )

si_text1 = system_prompt_v4

model = "gemini-2.0-flash-001"
contents = [
    types.Content(
        role="user",
        parts=[
        types.Part.from_text(text="""mèo siêu anh hùng""")
        ]
    )
]


class Recipe(BaseModel):
  recipe_name: str
  ingredients: list[str]

generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    system_instruction=[types.Part.from_text(text=si_text1)],
    response_schema= list[Recipe],
)


response = client.models.generate_content(
    model = model,
    contents = contents,
    config = generate_content_config,
    )

print(response.text)

ValueError: Unsupported schema type.

In [ ]:
class Recipe(BaseModel):
  recipe_name: str
  ingredients: list[str]
  
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='List a few popular cookie recipes. Be sure to include the amounts of ingredients.',
    config={
        'response_mime_type': 'application/json',
        'response_schema': list[Recipe],
        'system_instruction': types.Part.from_text(text="You are a cat. Your name is Neko. tell your name every time in recipe_name."),
    },
)
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_recipes: list[Recipe] = response.parsed

{
  "recipe_name": "Neko's Chocolate Chip Cookies",
  "ingredients": [
    "1 cup (2 sticks) unsalted butter, softened",
    "3/4 cup granulated sugar",
    "3/4 cup packed brown sugar",
    "1 teaspoon vanilla extract",
    "2 large eggs",
    "2 1/4 cups all-purpose flour",
    "1 teaspoon baking soda",
    "1 teaspoon salt",
    "2 cups chocolate chips"
  ]
}


In [23]:
json.loads(response.text)

{'recipe_name': "Neko's Chocolate Chip Cookies",
 'ingredients': ['1 cup (2 sticks) unsalted butter, softened',
  '3/4 cup granulated sugar',
  '3/4 cup packed brown sugar',
  '1 teaspoon vanilla extract',
  '2 large eggs',
  '2 1/4 cups all-purpose flour',
  '1 teaspoon baking soda',
  '1 teaspoon salt',
  '2 cups chocolate chips']}

In [15]:
from models import ComicScript
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='siêu nhân mèo',
    config={
        'response_mime_type': 'application/json',
        'response_schema': ComicScript,
        'system_instruction': types.Part.from_text(text=system_prompt_v4),
    },
)
# Use the response as a JSON string.
print(response.text)


ValidationError: 1 validation error for Schema
properties.characters.additionalProperties
  Extra inputs are not permitted [type=extra_forbidden, input_value={'$ref': '#/$defs/Character'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden

In [26]:
from google import genai
from google.genai import types
from io import BytesIO
from google.cloud import storage

genai_client= genai.Client(
      vertexai=True,
      project="thematic-land-451915-j3",
      location="us-central1",
  )

def generate_and_upload_public_image(bucket_name, blob_name, prompt='a portrait of a sheepadoodle wearing cape'):
    """Generates an image, uploads it to Google Cloud Storage, and returns the public URL."""
    try:
        response = genai_client.models.generate_images(
            model='imagen-3.0-generate-002',
            prompt=prompt,
            config=types.GenerateImagesConfig(number_of_images=1))

        image_bytes = response.generated_images[0].image.image_bytes

        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        blob.upload_from_string(image_bytes, content_type="image/png")

        return blob.public_url

    except Exception as e:
        print(f"Error: {e}")
        return None

# Replace with your actual values
bucket_name = 'bucket_comic'
blob_name = 'testai/sheepadoodle_cape.png'

public_url = generate_and_upload_public_image(bucket_name, blob_name)

if public_url:
    print(f"Public Image URL: {public_url}")

python(23507) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(23530) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(23546) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Public Image URL: https://storage.googleapis.com/bucket_comic/testai/sheepadoodle_cape.png


In [ ]:
# working code
import asyncio
from google.cloud import storage
import uuid
from google import genai
from google.genai import types
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

genai_client= genai.Client(
      vertexai=True,
      project="thematic-land-451915-j3",
      location="us-central1",
  )

def generate_image(prompt):
    """Generates an image synchronously using the genai client."""
    try:
        response = genai_client.models.generate_images(
            model='imagen-3.0-generate-002',
            prompt=prompt,
            config=types.GenerateImagesConfig(
                        number_of_images=1,
                        aspect_ratio="1:1",
                    )
            )

        return response.generated_images[0].image.image_bytes

    except Exception as e:
        logging.error(f"Error generating image: {e}", exc_info=True)
        return None

async def upload_image_async(image_bytes, bucket_name, prefix):
    """Uploads an image asynchronously to Google Cloud Storage."""
    try:
        blob_name = f"{prefix}{uuid.uuid4()}.png"

        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)

        if not bucket.exists():
            logging.error(f"Bucket {bucket_name} does not exist.")
            return "placeholder_image_url.jpg"

        blob = bucket.blob(blob_name)
        blob.upload_from_string(image_bytes, content_type="image/png")

        return blob.public_url

    except Exception as e:
        logging.error(f"Error uploading image: {e}", exc_info=True)
        return "placeholder_image_url.jpg"

async def main(comic_list, bucket_name, prefix):
    """Generates and uploads images for a comic list."""
    image_urls = []
    for page in comic_list['pages']:
        image_bytes = generate_image(page["image_prompt"])
        if image_bytes:
            url = await upload_image_async(image_bytes, bucket_name, prefix)
            image_urls.append(url)
        else:
            image_urls.append("placeholder_image_url.jpg")

    for scene, url in zip(comic_list['pages'], image_urls):
        scene["image_url"] = url

    return comic_list

# Example usage (replace with your comic_list and bucket details):
comic_list = {
    'pages': [
        {'image_prompt': 'a cat wearing a hat'},
        {'image_prompt': 'a dog playing fetch'},
        # ... more pages
    ]
}

bucket_name = "bucket_comic"
prefix = "gemini_image_"

# Run the asynchronous main function in Jupyter
result = await main(comic_list, bucket_name, prefix) #use await directly inside of notebook.
print(result)

python(24909) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(24929) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(24945) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(24969) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(24985) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


{'pages': [{'image_prompt': 'a cat wearing a hat', 'image_url': 'https://storage.googleapis.com/bucket_comic/gemini_image_bcd8fd72-96ae-42a4-8bcb-29208f871b76.png'}, {'image_prompt': 'a dog playing fetch', 'image_url': 'https://storage.googleapis.com/bucket_comic/gemini_image_82a1304a-a342-4b8a-8f22-2b528de72c9f.png'}]}
